In [1]:
import json
import re

In [19]:
def parse_session_to_states(session_data):
    state_sequence = ['S_Initial']
    conversation = session_data.get('conversation', [])

    for i, turn in enumerate(conversation):
        if turn['role'] == 'assistant':
            content = turn['content']

            previous_turn_role = conversation[i-1]['role'] if i > 0 else 'system'
            has_thought = '<Thought>' in content
            has_sub_question = '<Sub-Question>' in content
            has_search = '<Search>' in content
            has_end = '<End>' in content
            has_final_answer = 'Final Answer:' in content or '<Final Answer>:' in content
            
            previous_turn_role = conversation[i-1]['role'] if i > 0 else 'system'
            
            if state_sequence[-1] == 'S_Initial' and has_thought and has_search:
                state_sequence.append('S_Plan')
            
            elif previous_turn_role == 'user' and state_sequence[-1] == 'S_Retrieve':
                state_sequence.append('S_Assess')
            
            if state_sequence[-1] == 'S_Assess' and has_search and not has_end:
                state_sequence.append('S_Refine_Q')

            if has_search:
                if 'S_retrieve' != state_sequence[-1]:
                    state_sequence.append('S_Retrieve')
            
            if has_thought and has_end:
                if 'S_Generate' != state_sequence[-1]:
                    state_sequence.append('S_Generate')

            if has_final_answer:
                if 'S_Final' != state_sequence[-1]:
                    state_sequence.append('S_Final')
                    break
    
    final_sequence = []
    if state_sequence:
        final_sequence.append(state_sequence[0])
        for state in state_sequence[1:]:
            # if state != final_sequence[-1]:
                final_sequence.append(state)
    if 'S_Final' not in final_sequence:
        final_sequence.append('S_Fail')
    
    return final_sequence


In [21]:
file = '../result/omnisearch/output_omni.jsonl'
log_file = '../result/omnisearch/automaton.jsonl'
with open(file, 'r', encoding='utf-8') as f, \
    open(log_file, 'w', encoding='utf-8') as g:
    for line in f:
        data = json.loads(line)
        sequence = parse_session_to_states(data)
        g.write(json.dumps({'id': data['session_id'], 'sequence': sequence}, ensure_ascii=False) + '\n')
        print(f"Session {data['session_id']}: {' ->'.join(sequence)}")

Session 3a2b69dc-7833-4c79-96a5-1d74650e2057: S_Initial ->S_Retrieve ->S_Assess ->S_Final
Session 7b23bff8-7f17-41ee-8832-80ba361060ed: S_Initial ->S_Plan ->S_Retrieve ->S_Generate ->S_Final
Session a5f78985-9afd-4a54-9c39-b9e815d2701c: S_Initial ->S_Plan ->S_Retrieve ->S_Final
Session 5e2c92fa-dcf2-4029-8d34-d80c5f9d1800: S_Initial ->S_Plan ->S_Retrieve ->S_Assess ->S_Final
Session 25342fdd-7d87-4b50-b51f-7265425b1fbf: S_Initial ->S_Plan ->S_Retrieve ->S_Assess ->S_Refine_Q ->S_Retrieve ->S_Assess ->S_Refine_Q ->S_Retrieve ->S_Fail
Session 98e45741-0d6d-42e8-88dc-18dc5a8a92df: S_Initial ->S_Retrieve ->S_Assess ->S_Refine_Q ->S_Retrieve ->S_Final
Session ac452b51-483e-4884-89ba-439f82f50c97: S_Initial ->S_Plan ->S_Retrieve ->S_Assess ->S_Refine_Q ->S_Retrieve ->S_Fail
Session 1db6b6f7-1ada-4a31-8504-fb68ac8f2be8: S_Initial ->S_Plan ->S_Retrieve ->S_Generate ->S_Final
Session 2309b213-6ea7-4894-aa2a-f03f56126d91: S_Initial ->S_Retrieve ->S_Assess ->S_Final
Session 7365fb99-bfa5-49dc-841

# 2 创建转移矩阵

In [30]:
import pandas as pd
from typing import List

def build_transition_matrix(paths: List[List[str]]):
    states = set()
    for path in paths:
        states.update(path)
    sorted_states = sorted(list(states))

    num_states = len(sorted_states)
    count_matrix = pd.DataFrame(
        0,
        index = sorted_states,
        columns=sorted_states
    )

    for path in paths:
        for from_state, to_state in zip(path, path[1:]):
            count_matrix.loc[from_state, to_state] += 1

    print(f"count Matrix:\n{count_matrix}")

    prob_matrix = pd.DataFrame(0.0, index=sorted_states, columns=sorted_states)

    row_sums = count_matrix.sum(axis=1)
    transient_mask = row_sums > 0
    transient_states = row_sums[transient_mask].index.tolist()

    prob_matrix.loc[transient_states] = count_matrix.loc[transient_states].div(row_sums[transient_states], axis=0)

    absorbing_states = [s for s in sorted_states if s not in transient_states]

    for state in absorbing_states:
        prob_matrix.loc[state, state] = 1.0
    
    print("概率转移矩阵 (Probability Transition Matrix):")
    print(prob_matrix.round(3)) # 保留3位小数以便阅读
    return prob_matrix

In [31]:
with open(log_file, 'r', encoding='utf-8') as f:
    session_paths = []
    for line in f:
        data = json.loads(line)
        session_paths.append(data['sequence'])
    transition_matrix = build_transition_matrix(session_paths)

count Matrix:
            S_Assess  S_Fail  S_Final  S_Generate  S_Initial  S_Plan  \
S_Assess           0      94      461          33          0       0   
S_Fail             0       0        0           0          0       0   
S_Final            0       0        0           0          0       0   
S_Generate         0       0      182           0          0       0   
S_Initial          0      10       36           4          0     691   
S_Plan             0       0        0           0          0       0   
S_Refine_Q         0       0        0           0          0       0   
S_Retrieve      1309     242      173         145          0       0   

            S_Refine_Q  S_Retrieve  
S_Assess           718           3  
S_Fail               0           0  
S_Final              0           0  
S_Generate           0           0  
S_Initial            0         457  
S_Plan               0         691  
S_Refine_Q           0         718  
S_Retrieve           0           0  
概率转移

In [40]:
import numpy as np
import pandas as pd

def analyze_markov_chain(prob_matrix: pd.DataFrame):
    absorbing_states = prob_matrix[prob_matrix.index == prob_matrix.columns].apply(lambda row: row[row.name] == 1.0, axis=1)
    absorbing_states = absorbing_states[absorbing_states].index.to_list()
    transient_states = [s for s in prob_matrix.index if s not in absorbing_states]

    Q = prob_matrix.loc[transient_states, transient_states].to_numpy()
    R = prob_matrix.loc[transient_states, absorbing_states].to_numpy()

    I = np.identity(Q.shape[0])
    N = np.linalg.inv(I-Q)

    N_df = pd.DataFrame(N, index=transient_states, columns=transient_states)
    print("--- 核心分析结果 ---")
    print("\n基本吸收矩阵 N (Fundamental Matrix):")
    print(N_df.round(3))
    print("\n解释: N[i, j] 代表从状态 i 出发，在被吸收前，预期会访问状态 j 的次数。")
    
    expected_steps = N.sum(axis=1)

    expected_steps_df = pd.DataFrame(expected_steps, index=transient_states, columns=['Expected_Steps'])
    print("\nKPI 1: 系统效率 (预期总步数)")
    print(expected_steps_df.round(3))

    absorption_probs = N @ R
    absorption_probs_df = pd.DataFrame(absorption_probs, index=transient_states, columns=absorbing_states)

    print("\nKPI 2: 系统可靠性 (吸收概率)")
    print(absorption_probs_df.round(3))

    loop_states = ['S_Assess', 'S_Refine_Q', 'S_Retrieve']
    # 确保只选择存在于N_df中的状态
    actual_loop_states = [s for s in loop_states if s in N_df.columns]
    
    if actual_loop_states:
        loop_time = N_df[actual_loop_states].sum(axis=1)
        loop_time_df = pd.DataFrame(loop_time, index=transient_states, columns=['Loop_Time'])
        print("\nKPI 3: “死循环”量化 (在修正循环中的预期停留步数)")
        print(loop_time_df.round(3))

    return {
        "N_matrix": N_df,
        "expected_steps": expected_steps_df,
        "absorption_probs": absorption_probs_df,
        "loop_time": loop_time_df if actual_loop_states else None
    }


In [41]:
analysis_results = analyze_markov_chain(transition_matrix)

--- 核心分析结果 ---

基本吸收矩阵 N (Fundamental Matrix):
            S_Assess  S_Generate  S_Initial  S_Plan  S_Refine_Q  S_Retrieve
S_Assess       1.628       0.111        0.0   0.000       0.893       0.897
S_Generate     0.000       1.000        0.0   0.000       0.000       0.000
S_Initial      1.093       0.152        1.0   0.577       0.599       1.560
S_Plan         1.140       0.155        0.0   1.000       0.625       1.628
S_Refine_Q     1.140       0.155        0.0   0.000       1.625       1.628
S_Retrieve     1.140       0.155        0.0   0.000       0.625       1.628

解释: N[i, j] 代表从状态 i 出发，在被吸收前，预期会访问状态 j 的次数。

KPI 1: 系统效率 (预期总步数)
            Expected_Steps
S_Assess             3.528
S_Generate           1.000
S_Initial            4.981
S_Plan               4.549
S_Refine_Q           4.549
S_Retrieve           3.549

KPI 2: 系统可靠性 (吸收概率)
            S_Fail  S_Final
S_Assess     0.233    0.767
S_Generate   0.000    1.000
S_Initial    0.289    0.711
S_Plan       0.293    0.707
S_Ref

# 基本吸收矩阵的信息

1. S_Initial：
   1. S_Retrieval > 1说明为迭代式RAG，平均触发不止一次检索；
   2. S_Plan < 1，应检查没有Plan直接检索的情况，斟酌其合理性；
   3. S_refine_Q：超过一半的任务需要修正问题，说明对问题的修正的改进是系统花费较多时间的地方；
   4. S_Assess评估次数略小于检索次数：没有经过评估就直接生成
2. Plan 和Refine_Q数据相似度特别高，这可能暗示，无论是初始规划后，还是重构查询后，系统后续触发的检索行为模式是类似的。这可能揭示了系统的一个内在行为模式。
3. 系统进入S_Refine_Q之后，平均1.6次进入S_Refine_Q，说明具有循环